Install and initialize chromadb

In [1]:
import chromadb

Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 4


LlamaIndex Loader (Docs)

In [2]:
from llama_index.core import SimpleDirectoryReader

docs_dir = "docs"
documents = SimpleDirectoryReader(docs_dir).load_data()

LlamaIndex Chunking (+ Enrich metadata)

In [3]:
def extract_kv_metadata(text):
    metadata = {}
    current_list_key = None
    # Split lines, remove leading/trailing whitespace
    for line in text.splitlines():
        line = line.strip()
        # Check for main key-value pair
        if (line.startswith("- ") and ":" in line[2:]):
            item = line[2:]  # Remove leading "- "
            key, value = item.split(":", 1)
            metadata[key.strip().lower()] = value.strip()
            current_list_key = key.strip().lower() if value.strip() == "" else None
        # Check for inner list under previous key (like under "Languages" or "Contact")
        elif (line.startswith("- ") and current_list_key):
            value = line[2:].strip()
            if current_list_key not in metadata or not isinstance(metadata[current_list_key], list):
                metadata[current_list_key] = []
            metadata[current_list_key].append(value)
        # Also handle non-bullet "Key: Value"
        elif ":" in line and not line.startswith("#"):
            key, value = line.split(":", 1)
            metadata[key.strip().lower()] = value.strip()
            current_list_key = None
        else:
            current_list_key = None  # reset if not a valid list item
    return metadata


In [4]:
from llama_index.core.node_parser import MarkdownNodeParser

# Chunk and parse with MarkdownNodeParser
parser = MarkdownNodeParser(
    include_metadata=True,              # Attach section header path as metadata
    include_prev_next_rel=True,         # Good for context awareness
    header_path_separator=" > ",        # Separator for hierarchy in metadata
)

chunked_documents = parser.get_nodes_from_documents(documents) # in this case, we only have portfolio.md

# Create and attach metadata to chunked documents
for node in chunked_documents:
    kv_meta = extract_kv_metadata(node.get_content())
    node.metadata = {**node.metadata, **kv_meta}  # merge with header_path, etc.

    # debug view:
    print(node.metadata)
    print(node.get_content() + "\n")

{'file_path': 'c:\\Users\\aryan\\OneDrive - Thomas More\\FoolingAround\\portfolioRAG\\docs\\portfolio.md', 'file_name': 'portfolio.md', 'file_type': 'text/markdown', 'file_size': 11103, 'creation_date': '2025-10-12', 'last_modified_date': '2025-10-12', 'header_path': ' > '}
# Aryan Shah Portfolio

{'file_path': 'c:\\Users\\aryan\\OneDrive - Thomas More\\FoolingAround\\portfolioRAG\\docs\\portfolio.md', 'file_name': 'portfolio.md', 'file_type': 'text/markdown', 'file_size': 11103, 'creation_date': '2025-10-12', 'last_modified_date': '2025-10-12', 'header_path': ' > Aryan Shah Portfolio\r > ', 'first name': 'Aryan', 'last name': 'Shah', 'age': '20', 'location': 'Antwerp, Belgium', 'portfolio': 'https://l145.be/'}
## Personal Information

- First Name: Aryan
- Last Name: Shah
- Age: 20
- Location: Antwerp, Belgium
- Portfolio: https://l145.be/

{'file_path': 'c:\\Users\\aryan\\OneDrive - Thomas More\\FoolingAround\\portfolioRAG\\docs\\portfolio.md', 'file_name': 'portfolio.md', 'file_type

Connect to ChromaDB (Dev server + Cloud)

In [6]:
# Dev server

client = chromadb.HttpClient(host="localhost", port=8000)

2025-10-13 22:02:26,996 - WARNING - Python-dotenv could not parse statement starting at line 4
2025-10-13 22:02:26,997 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-10-13 22:02:27,210 - INFO - HTTP Request: GET http://localhost:8000/api/v2/auth/identity "HTTP/1.1 200 OK"
2025-10-13 22:02:27,212 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-10-13 22:02:27,421 - INFO - HTTP Request: GET http://localhost:8000/api/v2/tenants/default_tenant "HTTP/1.1 200 OK"
2025-10-13 22:02:27,425 - INFO - HTTP Request: GET http://localhost:8000/api/v2/tenants/default_tenant/databases/default_database "HTTP/1.1 200 OK"


In [7]:
# Wipes the old collection to ensure fresh data
# NUKES ENTIRE EXISTING COLLECTION, USE WITH CAUTION
client.delete_collection(name="aryan_portfolio") 

2025-10-13 22:02:30,126 - INFO - HTTP Request: DELETE http://localhost:8000/api/v2/tenants/default_tenant/databases/default_database/collections/aryan_portfolio "HTTP/1.1 200 OK"


In [8]:
# Dev server
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

collection = client.get_or_create_collection("aryan_portfolio")

local_vector_store = ChromaVectorStore(
    chroma_collection=collection
)

storage_context = StorageContext.from_defaults(vector_store=local_vector_store)

2025-10-13 22:02:44,443 - INFO - HTTP Request: POST http://localhost:8000/api/v2/tenants/default_tenant/databases/default_database/collections "HTTP/1.1 200 OK"


In [ ]:
# Cloud server
from llama_index.vector_stores.chroma import ChromaVectorStore
import os

tenant = os.environ.get("CHROMA_TENANT")
database = os.environ.get("CHROMA_DATABASE")

if tenant and database:
    cloud_vector_store = ChromaVectorStore(
        collection_name="aryan_portfolio",
        chroma_api_impl="rest",
        chroma_api_impl_options={
            "host": "api.trychroma.com", # f"{tenant}.{database}.chroma.com"
            "port": 8000,
            "ssl": True,
            "api_key": os.environ.get("CHROMA_API_KEY")
        }
    )

else:
    print("tenant and/or database environment variable not found!")

LlamaIndex Indexing

In [9]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding("all-MiniLM-L6-v2")

index = VectorStoreIndex(chunked_documents, storage_context=storage_context, embed_model=embed_model)

2025-10-13 22:04:11,078 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-10-13 22:04:13,846 - INFO - 2 prompts are loaded, with the keys: ['query', 'text']
2025-10-13 22:04:15,570 - INFO - HTTP Request: GET http://localhost:8000/api/v2/pre-flight-checks "HTTP/1.1 200 OK"
2025-10-13 22:04:15,750 - INFO - HTTP Request: POST http://localhost:8000/api/v2/tenants/default_tenant/databases/default_database/collections/2a54dd59-3e59-49c5-bcc6-4bc94bdc507f/add "HTTP/1.1 201 Created"


LlamaIndex Retriever

In [15]:
retriever = index.as_retriever(similarity_top_k=5)

In [23]:
# test retriever

results = retriever.retrieve("what are your achievements")

for item in results:
    node = item.node
    print("Score:", item.score)
    print("ID:", node.id_)
    print("Metadata:", node.metadata)
    print("Text:", getattr(node, "get_content", lambda: node.get_content())())
    print("---")

2025-10-13 22:13:07,018 - INFO - HTTP Request: POST http://localhost:8000/api/v2/tenants/default_tenant/databases/default_database/collections/2a54dd59-3e59-49c5-bcc6-4bc94bdc507f/query "HTTP/1.1 200 OK"


Score: 0.24465243703121758
ID: e5e4046e-45ca-4491-ad6d-3d5ee46e3661
Metadata: {'file_path': 'c:\\Users\\aryan\\OneDrive - Thomas More\\FoolingAround\\portfolioRAG\\docs\\portfolio.md', 'file_name': 'portfolio.md', 'file_type': 'text/markdown', 'file_size': 11103, 'creation_date': '2025-10-12', 'last_modified_date': '2025-10-12', 'header_path': ' > Aryan Shah Portfolio\r > Education\r > ', 'start year': '2023', 'end year': '2026', 'overall grade': '80%', 'description': 'Coursework includes web development, software development, networking, devops, databases, data analysis, data science, data engineering, machine learning, AI, and cybersecurity.'}
Text: ### Bachelor's in Data Science, Protection & Security – Thomas More University of Applied Sciences

- Start Year: 2023
- End Year: 2026
- Overall Grade: 80%
- Description: Coursework includes web development, software development, networking, devops, databases, data analysis, data science, data engineering, machine learning, AI, and cyber

In [27]:
# try chroma query

res = collection.query(
    query_texts=["what programming languages do you know"]
)

print(res['documents'])

2025-10-13 22:23:34,164 - INFO - HTTP Request: POST http://localhost:8000/api/v2/tenants/default_tenant/databases/default_database/collections/2a54dd59-3e59-49c5-bcc6-4bc94bdc507f/query "HTTP/1.1 200 OK"


[['### Holy Hack Hackathon 2025\r\n\r\n- Role: Frontend Developer\r\n- Date: March 2025\r\n- Position: Participation\r\n- Technologies: React, JavaScript, Python, Git, GitHub, Figma, Yahoo Finance API\r\n- Description: Built a web application within 24 hours to tailor a visual roadmap for various financial goals with real-time market history data and Monte Carlo simulation for investment strategies.', "#### Trump AI Features\r\n\r\n- Trump-style Responses: Engages users with witty replies mimicking Donald Trump's speech patterns.\r\n- Groq AI Integration: Leverages advanced AI capabilities for natural language understanding.\r\n- User-friendly Interface: Simple and intuitive design for seamless interactions.\r\n- Multi-platform Support: Accessible via web and mobile devices.", "### Trump AI\r\n\r\n- Description: A humorous AI chatbot that responds in the style of Donald Trump using Groq AI's llama-3.3-70b-versatile model.\r\n- Technologies: React, TypeScript, Express, NodeJS, Groq AI, 

Add LLM layer

In [11]:
from groq import Groq
import os
from dotenv import load_dotenv

Initialize Groq

In [12]:
load_dotenv()

client = Groq(
    api_key=os.environ.get("GROQ_KEY"),
)

Python-dotenv could not parse statement starting at line 4


Use user query for semantic search

In [35]:
user_query = input("What would you like to know about Aryan?: ")

results = collection.query(
    query_texts=[user_query],
    n_results=5
)

print(results)

{'ids': [['personal_info', 'education_0', 'hackathon_0', 'hackathon_1_tech_3', 'hackathon_1']], 'distances': [[1.2531397, 1.2962006, 1.2998523, 1.3314855, 1.3482904]], 'embeddings': None, 'metadatas': [[{'type': 'personal_info'}, {'type': 'education', 'degree': "Bachelor's in Data Science, Protection & Security", 'institution': 'Thomas More University of Applied Sciences'}, {'role': 'Frontend Developer', 'position': 'participation', 'type': 'hackathon', 'name': 'Holy Hack 2025'}, {'hackathon': 'TCS Design Thinking Hackathon 2025', 'type': 'technology'}, {'name': 'TCS Design Thinking Hackathon 2025', 'position': '2nd Place', 'type': 'hackathon', 'role': 'UI/UX Designer & Prototype Developer'}]], 'documents': [['My name is Aryan Shah, 20 years old, based in Antwerp, Belgium. My career focus is Frontend/Full-Stack Development.', "I studied Bachelor's in Data Science, Protection & Security at Thomas More University of Applied Sciences (2023 - 2026). Coursework includes web development, sof

Prompting AI

In [36]:
stream = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """
                You will be Aryan and answer based on Aryan's portfolio with the following guidelines:

                You will use the following writing style for all responses:
                - True and authentic information only, no made up information.
                - Direct, confident, and concise communication
                - Professional yet conversational tone
                - Clear structure with logical flow between ideas
                - Emphasis on ownership and impact ("I took full ownership", "I built")
                - Explicit alignment of skills/interests with the context
                - Practical focus on value creation and application
                - Active voice and strong action verbs
                - Connecting personal experiences to broader relevance
                - Concise paragraphs that get straight to the point
                - Natural transitions between technical and personal aspects
                
                Example (Not to be used literally): As for my professional experience, it was mostly frontend based, where I 
                    took full ownership over a web-based tool used by various departments 
                    within the company. Since it was a specialized tool, communication was 
                    key to designing it in a way which is ideal for the use case. I built out the 
                    design and made tests to ensure the web application works as expected. 
                    This development cycle is not only related to the web, and can be carried 
                    over to any stack.

                Important: If the information is not available in the context, respond with saying something like that it's not available and guide them to ask something relevant to the portfolio instead. Say this based on the user query in a funny (somewhat normally like a regular 20 year old student) way, flip the table smoothly.
                VERY IMPORTANT: FACT CHECK THE INFORMATION, IF THE INFORMATION IS NOT IN THE CONTEXT, DO NOT INCLUDE IT.
            """
        },
        {
            "role": "user",
            "content": user_query + "\n\nUse the following context from Aryan's portfolio to answer the question:\n" + str(results),
        }
    ],
    model="openai/gpt-oss-20b",
    stream=True,
)

# Print the incremental deltas returned by the LLM.
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


I’m Aryan Shah, a front‑end / full‑stack developer who already built a real‑time web app from scratch in 24 hours during Holy Hack 2025.  I wrote the client side, wired it to live market data, and ran Monte Carlo simulations—all in a modern JavaScript stack that you’ll find aligns perfectly with React.

**React‑style front‑end chops**  
- In the hackathon I took full ownership of the UI, crafting reusable components and responsive layouts that delivered a polished user experience.  
- I’m comfortable with state‑management patterns, component lifecycle, and CSS‑in‑JS—all the core skills that React teams rely on.

**API‑side experience**  
- The same project pulled data from external financial APIs, parsed JSON, and fed the results into the front‑end in real time.  
- My coursework in web development and networking gave me hands‑on practice designing REST endpoints, handling authentication, and integrating third‑party services.

**Broader foundation**  
- Bachelor’s in Data Science, Prot